### Enable debug logging

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)


### Initialise the connection to the Aircloak server

In [2]:
from explorer.connection import AircloakConnection

aircloak = AircloakConnection(dbname='GiveMeSomeCredit')

DEBUG:root:Connecting to Aircloak: user=daniel-613C7ADF4535BB56DBCD, host=attack.aircloak.com, port=9432, dbname=GiveMeSomeCredit


### Import the Explorer and launch the exploration

In [3]:
from explorer.numeric_explorer import NumericColumnExplorer

e = NumericColumnExplorer(aircloak_connection=aircloak, table='loans', column='MonthlyIncome')


DEBUG:root:Sending query: 
        SELECT
            min("MonthlyIncome")
        ,   max("MonthlyIncome")
        ,   avg("MonthlyIncome")
        ,   count(*)
        ,   count_noise(*)
        FROM "loans"
    
DEBUG:root:Sending query: 
        SELECT
            "MonthlyIncome"
        ,   count(*)
        FROM "loans"
        WHERE "MonthlyIncome" IS NOT NULL
        GROUP BY 1
    
DEBUG:root:Sending query: 
    SELECT
        bucket("MonthlyIncome" by 10000) as "bucket_10000"
, bucket("MonthlyIncome" by 500) as "bucket_500"
, bucket("MonthlyIncome" by 2000) as "bucket_2000"
    ,   count(*)
    ,   count_noise(*)
    ,   min("MonthlyIncome")
    ,   max("MonthlyIncome")
    ,   avg("MonthlyIncome")
    FROM "loans"
    WHERE "MonthlyIncome" IS NOT NULL
    GROUP BY GROUPING SETS (1, 2, 3)
    
DEBUG:root:Received query results, processing...
DEBUG:root:... finished processing query results.


### Explore some additional buckets

In [4]:
e.explore({1,10,50})

DEBUG:root:Sending query: 
    SELECT
        bucket("MonthlyIncome" by 1) as "bucket_1"
, bucket("MonthlyIncome" by 10) as "bucket_10"
, bucket("MonthlyIncome" by 50) as "bucket_50"
    ,   count(*)
    ,   count_noise(*)
    ,   min("MonthlyIncome")
    ,   max("MonthlyIncome")
    ,   avg("MonthlyIncome")
    FROM "loans"
    WHERE "MonthlyIncome" IS NOT NULL
    GROUP BY GROUPING SETS (1, 2, 3)
    
DEBUG:root:Received query results, processing...
DEBUG:root:... finished processing query results.


In [5]:
print(f'suppressed rows: {sorted(e._suppressed_counts.items())}, total rows: {e._count}')

suppressed rows: [(0, 13818), (1, 13818), (10, 1525), (50, 580), (500, 170), (2000, 62), (10000, 36)], total rows: 150000


In [26]:
### Extract to pandas dataframe for analysis

In [6]:
import pandas as pd

df = pd.DataFrame(**e.extract_to_dataframe())
df = df.set_index(['bucket_size'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8053 entries, 10000 to 50
Data columns (total 6 columns):
lower_bound    8053 non-null float64
count          8053 non-null int64
count_noise    7351 non-null float64
min            3185 non-null float64
max            3185 non-null float64
avg            7351 non-null float64
dtypes: float64(5), int64(1)
memory usage: 440.4 KB


In [7]:
df.head()

,lower_bound,count,count_noise,min,max,avg
bucket_size,,,,,,
10000,80000.0,26,1.4,78700.0,91302.0,84942.479711
10000,150000.0,5,1.4,NaN,NaN,150456.498912
10000,70000.0,48,1.4,69019.0,80385.0,73479.089119
10000,50000.0,94,1.4,49104.0,61425.0,54136.330838
10000,30000.0,371,1.4,29517.0,40973.0,33325.419025


### Plot column distribution as a bar chart

In [8]:
import plotly.express as px

px.bar(df.loc[[500]], x='lower_bound', y='count', hover_data=['lower_bound', 'count', 'min', 'max', 'count_noise'])

### Get a sum of the counts by bucket size

In [9]:
df['count'].groupby('bucket_size').sum()

bucket_size
1        105120
10       118677
50       119646
500      120064
2000     120214
10000    120231
Name: count, dtype: int64